In [1]:
# IMPORTS
from preprocessing_utils import *
from global_vars import *
from random_forest_model import *
from GAN_network_vanila import *
import pandas as pd
from typing import Tuple

In [ ]:
# get number of unique values in each column
def get_unique_values(df: pd.DataFrame) -> Tuple[dict, dict]:
    pass

In [2]:
data_df, df_attributes = read_arff_as_dataframe(G_CREDIT_PATH)